In [10]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [5]:
!cp "/content/sample_data/CLAN_data_cleaned.csv" .

In [6]:
!pip install transformers datasets evaluate torch tqdm
!pip install rouge-score nltk bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=aee190f0d272ed5f00ead8a868917e58cf8b782e3b712be510ced74d81b20d10
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


Imports

In [7]:
import torch
import pandas as pd
import torch.nn as nn
from transformers import T5Tokenizer, T5ForConditionalGeneration, get_scheduler
from torch.optim import AdamW
from tqdm import tqdm
from datasets import load_dataset
from evaluate import load
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [8]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [10]:
df = pd.read_csv("/content/sample_data/CLAN_data_cleaned.csv")
df

,PID,Social Media Post,Normalized Claim
0,1,"president ""biden's plan would mean america's s...",biden s energy plan would get rid of seniors a...
1,2,important announcement coronavirus last evenin...,"if someone with the new coronavirus sneezes, i..."
2,3,heart is delighted to hear,heart is delighted to hear
3,4,an allowed appeal is one where the initial ref...,the vast majority of people coming across the ...
4,5,warm water therapy dr. d. mensah asare says th...,a widely popular social media post claims that...
...,...,...,...
2285,2798,"foh with that weak malala slander, twitter fin...",malala yousafzai has been silent about the cri...
2286,2802,the plight of hindu girls madly in love with m...,the plight of hindu girls madly in love with m...
2287,2804,i am hearing that james comey has 50 counts of...,james comey has 50 counts of treason. and john...
2288,2806,dr. lonnie herman reviews how your food is pre...,irradiated food causes cancer


Load Datasets

In [11]:
# Load the cleaned CLAN dataset
df = pd.read_csv("/content/CLAN_data_cleaned.csv")

# Select necessary columns
df = df[["Social Media Post", "Normalized Claim"]].dropna()

# Rename for easier reference
df.columns = ["input_text", "target_text"]

# Check dataset sample
print(df.head())

                                          input_text  \
0  president "biden's plan would mean america's s...   
1  important announcement coronavirus last evenin...   
2                         heart is delighted to hear   
3  an allowed appeal is one where the initial ref...   
4  warm water therapy dr. d. mensah asare says th...   

                                         target_text  
0  biden s energy plan would get rid of seniors a...  
1  if someone with the new coronavirus sneezes, i...  
2                         heart is delighted to hear  
3  the vast majority of people coming across the ...  
4  a widely popular social media post claims that...  


Tokenization

In [12]:
# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization
df["input_ids"] = df["input_text"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True, max_length=512, return_tensors="pt")["input_ids"].squeeze(0))
df["labels"] = df["target_text"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True, max_length=128, return_tensors="pt")["input_ids"].squeeze(0))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Created Datasets & Dataloader

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader

class ClaimDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.data.iloc[idx]["input_ids"], dtype=torch.long),
            "labels": torch.tensor(self.data.iloc[idx]["labels"], dtype=torch.long),
        }

# Create Dataset
train_dataset = ClaimDataset(df)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


Load T5 Model

In [17]:
from transformers import T5ForConditionalGeneration, get_scheduler

# Enable GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load T5 model on GPU
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)


Using device: cuda


Optimizer

In [18]:
# Optimizer & Scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_dataloader) * 5
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

Training Loop

In [19]:
from tqdm import tqdm
import torch.nn as nn

# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")

    for batch in progress_bar:
        # Move batch data to GPU
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} Loss: {avg_loss}")


Epoch 1:   0%|          | 0/286 [00:00<?, ?it/s]<ipython-input-16-17b084072012>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(self.data.iloc[idx]["input_ids"], dtype=torch.long),
<ipython-input-16-17b084072012>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(self.data.iloc[idx]["labels"], dtype=torch.long),
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Epoch 1: 100%|██████████| 286/286 [01:37<00:00,  2.92it/s, Loss=0.188]


Epoch 1 Loss: 0.9506289193055013


Epoch 2: 100%|██████████| 286/286 [01:39<00:00,  2.88it/s, Loss=0.492]


Epoch 2 Loss: 0.6608988491596876


Epoch 3: 100%|██████████| 286/286 [01:39<00:00,  2.87it/s, Loss=0.148]


Epoch 3 Loss: 0.5999436210189666


Epoch 4: 100%|██████████| 286/286 [01:39<00:00,  2.87it/s, Loss=0.785]


Epoch 4 Loss: 0.579877832240158


Epoch 5: 100%|██████████| 286/286 [01:39<00:00,  2.89it/s, Loss=0.628]

Epoch 5 Loss: 0.5693822305623468


Evaluate BLUE , BERT & ROGUE

In [21]:
from evaluate import load

# Load Metrics
rouge = load("rouge")
bleu = load("bleu")
bertscore = load("bertscore")

# Evaluation Function
def evaluate(model, dataloader):
    model.eval()
    predictions = []  # Corrected
    references = []  # Corrected

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            outputs = model.generate(input_ids)
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            predictions.extend(decoded_preds)
            references.extend(decoded_labels)

    # Compute Metrics
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    bleu_scores = bleu.compute(predictions=predictions, references=references)
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="en")

    return rouge_scores, bleu_scores, bert_scores


# Run Evaluation
rouge_scores, bleu_scores, bert_scores = evaluate(model, train_dataloader)

# Print Scores
print("ROUGE-L:", rouge_scores["rougeL"])
print("BLEU-4:", bleu_scores["bleu"])
print("BERTScore:", sum(bert_scores["f1"]) / len(bert_scores["f1"]))


Evaluating:   0%|          | 0/286 [00:00<?, ?it/s]<ipython-input-16-17b084072012>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(self.data.iloc[idx]["input_ids"], dtype=torch.long),
<ipython-input-16-17b084072012>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(self.data.iloc[idx]["labels"], dtype=torch.long),
Evaluating: 100%|██████████| 286/286 [01:42<00:00,  2.78it/s]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-L: 0.2349669393434074
BLEU-4: 0.11034054879077468
BERTScore: 0.8269583252583194


Save the Model

In [11]:
model.save_pretrained("/content/drive/MyDrive/ClaimNormalization_T5_model")
tokenizer.save_pretrained("/content/drive/MyDrive/ClaimNormalization_T5_model")

# Create a directory to store the model
save_path = "/content/drive/MyDrive/ClaimNormalization_T5_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Save model state
torch.save(model.state_dict(), f"{save_path}/checkpoint_epoch_5.pth")

print(f"Model saved successfully at {save_path}")

NameError: name 'model' is not defined

Run Inference

In [12]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model and tokenizer from saved directory
load_path = "/content/drive/MyDrive/ClaimNormalization_T5_model"
tokenizer = T5Tokenizer.from_pretrained(load_path)
model = T5ForConditionalGeneration.from_pretrained(load_path).to(device)

# Function for user input inference
def generate_claim(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512).to(device)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Take user input for testing
user_text = input("Enter a social media post: ")
generated_claim = generate_claim(user_text)
print("\n🔹 Normalized Claim:", generated_claim)


Enter a social media post: BREAKING🚨: Scientists PROVE EARTH IS FLAT!!! 🌎🤯 #WakeUp

🔹 Normalized Claim: : Scientists PROVE EARTH IS FLAT!!!
